In [1]:
import os
import urllib.request
from bs4 import BeautifulSoup
import re
from datetime import datetime
from pytz import timezone
import pandas as pd
import csv
import time

def iso2utc(iso):
    try:
        return timezone('UTC').localize(pd.to_datetime(iso))
    except:
        return None

def url_html(url):
    req = urllib.request.Request(url)
    res = urllib.request.urlopen(req)
    html = res.read()
    return html

In [8]:
root_url = 'https://matome.naver.jp'
html = url_html(root_url)
soup = BeautifulSoup(html,'html.parser')
for li in soup.select('.mdTopMTMList01List')[0].select('li'):
    text = li.select('.mdTopMTMList01ItemTtl')[0].text
    url = root_url + li.select('.mdTopMTMList01ItemTtl')[0].a.get('href')
    img = li.find('img').get('src')
    print(text, url, img)

竹野内豊も初主演！秋の「ＮＨＫドラマ」が楽しそう https://matome.naver.jp/odai/2150063735056186801 https://rr.img.naver.jp/mig?src=http%3A%2F%2Fimgcc.naver.jp%2Fkaze%2Fmission%2FUSER%2F20170721%2F71%2F7306501%2F144%2F478x478x809046d32283b089634cb40f.jpg&twidth=60&theight=60&qlt=80&res_format=jpg&op=sc
イモトアヤコだけじゃない‥最近､女芸人たちの美人化が止まらない… https://matome.naver.jp/odai/2150063904056983501 https://rr.img.naver.jp/mig?src=http%3A%2F%2Fimgcc.naver.jp%2Fkaze%2Fmission%2FUSER%2F20170722%2F31%2F3056261%2F5%2F240x240xc886a1217dd0fd796ebfd970.jpg&twidth=60&theight=60&qlt=80&res_format=jpg&op=sc
青いストライプのシャツを着てにっこり…４歳になったジョージ王子… https://matome.naver.jp/odai/2150069296076264701 https://rr.img.naver.jp/mig?src=https%3A%2F%2Fmedia.gettyimages.com%2Fphotos%2Fthis-photograph-is-provided-to-you-strictly-on-condition-that-you-picture-id820429368%3Fb%3D1%26k%3D6%26m%3D820429368%26s%3D612x612%26w%3D0%26h%3DMTyzZNqgA806rzfYQACV7LuGtDC65jaIeKwa2T6NZ58%3D&twidth=60&theight=60&qlt=80&res_format=jpg&op=sc
パクチストをうならせる!?パクチーの新メニュー新商品が美味しそ… ht

In [ ]:


def br2ln(soup):
    text = str(soup)
    text = re.sub('^(.+?)&gt;&gt;([0-9]+)<br>(.+?)$','\\1<anchor>\\2</anchor><text>\\3</text>',text)
    text = re.sub('<br\s*?>', '\n', text)
    text = re.sub('\n+', '\n', text)
    return BeautifulSoup(text,'html.parser')

def url_csv(url):
    max_col_nb = 6
    
    html = url_html(url)
    soup = BeautifulSoup(html,'html.parser')
    t = [ a.extract() for a in soup.select('ares')]
    t = [ e.extract() for e in soup.select('aside')]

    lines = []

    a = soup.select('.article_time')[0].text.split()
    YYYY, mm, dd = a[0].split('-')
    HH, MM, SS = a[1].split(':')
    iso = YYYY+mm+dd+'T'+HH+MM+SS
    since = iso2utc(iso).isoformat()

    res0 = soup.select('.main_article_title')[0].text.split()[0]
    img = soup.select('.main_article_thumb')[0].select('img')[0].get('src')
    
    line = [str(0),'-1',res0,since,url,img]
    lines.append(line)

    for block in soup.select('.res_block'):
        if len(block.select('.news_quote')):
            res = '1'
            since = since
            news_quote = br2ln(block.select('.news_quote')[0])
            text = news_quote.text
            anchor = '0'
            imgs = [ e.get('src') for e in news_quote.select('img') ]

        if len(block.select('.res_info')):
            if not 'id' in block.attrs:
                res_body = br2ln(block.find_all(class_=re.compile('^res_body*'))[0])
                imgs = [ e.get('src') for e in res_body.select('img') ]
                lines[-1]+=imgs
                continue
            
            res = block.get('id').split('_')[1]

            date = block.select('.res_date')[0].text
            YYYY, mm, DD = date.split('(')[0].split('/')
            HH, MM, SS = date.split(')')[1].split()[0].split('.')[0].split(':')
            iso = YYYY+mm+dd+'T'+HH+MM+SS
            since = iso2utc(iso).isoformat()

            res_body = br2ln(block.find_all(class_=re.compile('^res_body*'))[0]) 
            text = res_body.text
            
            anchor = '0'
            if len(res_body.select('anchor')):
                anchor = res_body.find('anchor').text
                text = res_body.find('text').text

            imgs = [ e.get('src') for e in res_body.select('img') ]
            
        line = [str(res),str(anchor),text,since,url]+imgs
        lines.append(line)
        max_col_nb = max(max_col_nb, len(line))

    header = ['res','anchor','text','since','url','img']
    header+= ['img' for _ in range( max_col_nb - 6 ) ]

    return [header] + lines